<a href="https://colab.research.google.com/github/Ruby-L-i-n/CSDS234_FinalProject/blob/main/FeatureGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install textblob

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob
import re

In [7]:
track_df = pd.read_csv("/content/drive/Shareddrives/CSDS 234 Final Project/track_data/tracks_info_0-9999.csv")

track_df

,track_name,artist_name,artist_uri,genres,popularity,danceability,energy,key,loudness,mode,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Lose Control (feat. Ciara & Fat Man Scoop),Missy Elliott,spotify:artist:2wIVse2owClT7go1WT98tk,"['dance pop', 'hip hop', 'hip pop', 'neo soul'...",69,0.904,0.813,4,-7.105,0,...,0.0471,0.810,125.461,audio_features,0UaMYEvWZi0ZqiDOoHU3YI,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,https://api.spotify.com/v1/audio-analysis/0UaM...,226864,4
1,Toxic,Britney Spears,spotify:artist:26dSoYclwsYLMAKD3tpOr4,"['dance pop', 'pop']",79,0.774,0.838,5,-3.914,0,...,0.2420,0.924,143.040,audio_features,6I9VzXrHxO9rA9A5euc8Ak,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,https://api.spotify.com/v1/tracks/6I9VzXrHxO9r...,https://api.spotify.com/v1/audio-analysis/6I9V...,198800,4
2,Crazy In Love,Beyoncé,spotify:artist:6vWDO969PvNqNYHIOW5v0m,"['pop', 'r&b']",83,0.664,0.758,2,-6.583,0,...,0.0598,0.701,99.259,audio_features,0WqIKmW4BTrj3eJFmnCKMv,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,https://api.spotify.com/v1/tracks/0WqIKmW4BTrj...,https://api.spotify.com/v1/audio-analysis/0WqI...,235933,4
3,Rock Your Body,Justin Timberlake,spotify:artist:31TPClRtHm23RisEBtV3X7,"['dance pop', 'pop']",79,0.892,0.714,4,-6.055,0,...,0.0521,0.817,100.972,audio_features,1AWQoqb9bSvzTjaLralEkT,spotify:track:1AWQoqb9bSvzTjaLralEkT,https://api.spotify.com/v1/tracks/1AWQoqb9bSvz...,https://api.spotify.com/v1/audio-analysis/1AWQ...,267267,4
4,It Wasn't Me,Shaggy,spotify:artist:5EvFsr3kj42KNv97ZEnqij,"['pop rap', 'reggae fusion']",70,0.853,0.606,0,-4.596,1,...,0.3130,0.654,94.759,audio_features,1lzr43nnXAijIGYnCT8M8H,spotify:track:1lzr43nnXAijIGYnCT8M8H,https://api.spotify.com/v1/tracks/1lzr43nnXAij...,https://api.spotify.com/v1/audio-analysis/1lzr...,227600,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34437,Diamond Child,Jon D,spotify:artist:1r2kTJ27zuaEoXasQT5NDd,['channel pop'],39,0.669,0.228,2,-12.119,1,...,0.0944,0.402,83.024,audio_features,3uCHI1gfOUL5j5swEh0TcH,spotify:track:3uCHI1gfOUL5j5swEh0TcH,https://api.spotify.com/v1/tracks/3uCHI1gfOUL5...,https://api.spotify.com/v1/audio-analysis/3uCH...,189184,4
34438,I Don't Know,Big Words,spotify:artist:5HCypjplgh5uQezvBpOfXN,['australian r&b'],34,0.493,0.727,1,-5.031,1,...,0.1290,0.289,73.259,audio_features,0P1oO2gREMYUCoOkzYAyFu,spotify:track:0P1oO2gREMYUCoOkzYAyFu,https://api.spotify.com/v1/tracks/0P1oO2gREMYU...,https://api.spotify.com/v1/audio-analysis/0P1o...,263680,4
34439,The Answer,Allan Rayman,spotify:artist:0sHN89qak07mnug3LVVjzP,['canadian contemporary r&b'],48,0.702,0.524,7,-10.710,1,...,0.2980,0.265,140.089,audio_features,2oM4BuruDnEvk59IvIXCwn,spotify:track:2oM4BuruDnEvk59IvIXCwn,https://api.spotify.com/v1/tracks/2oM4BuruDnEv...,https://api.spotify.com/v1/audio-analysis/2oM4...,189213,4
34440,25.22,Jon Jason,spotify:artist:6Yv6OBXD6ZQakEljaGaDAk,[],2,0.509,0.286,8,-14.722,1,...,0.1310,0.259,121.633,audio_features,4Ri5TTUgjM96tbQZd5Ua7V,spotify:track:4Ri5TTUgjM96tbQZd5Ua7V,https://api.spotify.com/v1/tracks/4Ri5TTUgjM96...,https://api.spotify.com/v1/audio-analysis/4Ri5...,194720,4


In [197]:
def getSubjectivity(text):
  return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
  return TextBlob(text).sentiment.polarity

def getAnalysis(score, task="polarity"):
  if task == "subjectivity":
    if score < 1/3:
      return "low"
    elif score > 1/3:
      return "high"
    else:
      return "medium"
  else:
    if score < 0:
      return 'Negative'
    elif score == 0:
      return 'Neutral'
    else:
      return 'Positive'

def sentiment_analysis(df, text_col):
  # Perform sentiment analysis on text
  df['subjectivity'] = df[text_col].apply(getSubjectivity).apply(lambda x: getAnalysis(x,"subjectivity"))
  df['polarity'] = df[text_col].apply(getPolarity).apply(getAnalysis)
  return df

In [109]:
# Show result
sentiment = sentiment_analysis(track_df, "track_name")
sentiment

,track_name,artist_name,artist_uri,genres,popularity,danceability,energy,key,loudness,mode,...,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,subjectivity,polarity
0,Lose Control (feat. Ciara & Fat Man Scoop),Missy Elliott,spotify:artist:2wIVse2owClT7go1WT98tk,"['dance pop', 'hip hop', 'hip pop', 'neo soul'...",69,0.904,0.813,4,-7.105,0,...,125.461,audio_features,0UaMYEvWZi0ZqiDOoHU3YI,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,https://api.spotify.com/v1/audio-analysis/0UaM...,226864,4,low,Neutral
1,Toxic,Britney Spears,spotify:artist:26dSoYclwsYLMAKD3tpOr4,"['dance pop', 'pop']",79,0.774,0.838,5,-3.914,0,...,143.040,audio_features,6I9VzXrHxO9rA9A5euc8Ak,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,https://api.spotify.com/v1/tracks/6I9VzXrHxO9r...,https://api.spotify.com/v1/audio-analysis/6I9V...,198800,4,low,Neutral
2,Crazy In Love,Beyoncé,spotify:artist:6vWDO969PvNqNYHIOW5v0m,"['pop', 'r&b']",83,0.664,0.758,2,-6.583,0,...,99.259,audio_features,0WqIKmW4BTrj3eJFmnCKMv,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,https://api.spotify.com/v1/tracks/0WqIKmW4BTrj...,https://api.spotify.com/v1/audio-analysis/0WqI...,235933,4,high,Negative
3,Rock Your Body,Justin Timberlake,spotify:artist:31TPClRtHm23RisEBtV3X7,"['dance pop', 'pop']",79,0.892,0.714,4,-6.055,0,...,100.972,audio_features,1AWQoqb9bSvzTjaLralEkT,spotify:track:1AWQoqb9bSvzTjaLralEkT,https://api.spotify.com/v1/tracks/1AWQoqb9bSvz...,https://api.spotify.com/v1/audio-analysis/1AWQ...,267267,4,low,Neutral
4,It Wasn't Me,Shaggy,spotify:artist:5EvFsr3kj42KNv97ZEnqij,"['pop rap', 'reggae fusion']",70,0.853,0.606,0,-4.596,1,...,94.759,audio_features,1lzr43nnXAijIGYnCT8M8H,spotify:track:1lzr43nnXAijIGYnCT8M8H,https://api.spotify.com/v1/tracks/1lzr43nnXAij...,https://api.spotify.com/v1/audio-analysis/1lzr...,227600,4,low,Neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34437,Diamond Child,Jon D,spotify:artist:1r2kTJ27zuaEoXasQT5NDd,['channel pop'],39,0.669,0.228,2,-12.119,1,...,83.024,audio_features,3uCHI1gfOUL5j5swEh0TcH,spotify:track:3uCHI1gfOUL5j5swEh0TcH,https://api.spotify.com/v1/tracks/3uCHI1gfOUL5...,https://api.spotify.com/v1/audio-analysis/3uCH...,189184,4,low,Neutral
34438,I Don't Know,Big Words,spotify:artist:5HCypjplgh5uQezvBpOfXN,['australian r&b'],34,0.493,0.727,1,-5.031,1,...,73.259,audio_features,0P1oO2gREMYUCoOkzYAyFu,spotify:track:0P1oO2gREMYUCoOkzYAyFu,https://api.spotify.com/v1/tracks/0P1oO2gREMYU...,https://api.spotify.com/v1/audio-analysis/0P1o...,263680,4,low,Neutral
34439,The Answer,Allan Rayman,spotify:artist:0sHN89qak07mnug3LVVjzP,['canadian contemporary r&b'],48,0.702,0.524,7,-10.710,1,...,140.089,audio_features,2oM4BuruDnEvk59IvIXCwn,spotify:track:2oM4BuruDnEvk59IvIXCwn,https://api.spotify.com/v1/tracks/2oM4BuruDnEv...,https://api.spotify.com/v1/audio-analysis/2oM4...,189213,4,low,Neutral
34440,25.22,Jon Jason,spotify:artist:6Yv6OBXD6ZQakEljaGaDAk,[],2,0.509,0.286,8,-14.722,1,...,121.633,audio_features,4Ri5TTUgjM96tbQZd5Ua7V,spotify:track:4Ri5TTUgjM96tbQZd5Ua7V,https://api.spotify.com/v1/tracks/4Ri5TTUgjM96...,https://api.spotify.com/v1/audio-analysis/4Ri5...,194720,4,low,Neutral


In [188]:
def ohe_process(df, column, new_name):
  tf_df = pd.get_dummies(df[column])
  feature_names = tf_df.columns
  tf_df.columns = [new_name + "|" + str(i) for i in feature_names]
  tf_df.reset_index(drop = True, inplace = True)
  return tf_df

In [189]:
def subjectivity(df):
  subject_ohe = ohe_process(df, "subjectivity", "subjectivity") * 0.3
  return subject_ohe

In [190]:
def polarity(df):
  polarity_ohe = ohe_process(df, "polarity", "polarity") * 0.5
  return polarity_ohe

In [191]:
def genre_tfidf(df):
  tfidf = TfidfVectorizer()
  tfidf_track_matrix = tfidf.fit_transform(df['genres'])
  genre_df = pd.DataFrame(tfidf_track_matrix.toarray())
  genre_df.columns = ['genre' + '|' + i for i in tfidf.get_feature_names_out()]
  return genre_df

  # genre_df['genre|rap']

In [192]:
def popularity_normalization(df):
  popularity = df[['popularity']]
  scaler = MinMaxScaler()
  popularity_df = pd.DataFrame(scaler.fit_transform(popularity), columns = ['popularity'])
  return popularity_df

In [195]:
# audio features
def audio_features(df):
  #one-hot encoding
  key_ohe = ohe_process(df, "key", "key") * 0.5
  mode_ohe = ohe_process(df, "mode", "mode") * 0.5
  #normalization of other audio_features
  floats_cols = df.dtypes[df.dtypes == 'float64'].index.values #selects the audio feature colns
  floats = df[floats_cols].reset_index(drop = True)
  scaler = MinMaxScaler()
  floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns) * 0.2
  #putting them altogether
  audio_feats_df = pd.concat([key_ohe, mode_ohe, floats_scaled], axis=1)
  return audio_feats_df

audio_features(track_df)


,key|0,key|1,key|2,key|3,key|4,key|5,key|6,key|7,key|8,key|9,...,mode|1,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.182996,0.1626,0.168547,0.025156,0.006245,0.001401,0.00942,0.162325,0.114421
1,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,...,0.0,0.156680,0.1676,0.178715,0.023701,0.005000,0.005025,0.04840,0.185170,0.130453
2,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.134413,0.1516,0.170210,0.043659,0.000478,0.000000,0.01196,0.140481,0.090525
3,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.180567,0.1428,0.171892,0.029314,0.040361,0.000047,0.01042,0.163727,0.092087
4,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.172672,0.1212,0.176541,0.014823,0.011265,0.000000,0.06260,0.131062,0.086421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34437,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.135425,0.0456,0.152570,0.014345,0.159036,0.013065,0.01888,0.080561,0.075718
34438,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.099798,0.1454,0.175155,0.045114,0.017530,0.000000,0.02580,0.057916,0.066813
34439,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,...,0.5,0.142105,0.1048,0.157060,0.016486,0.066667,0.011116,0.05960,0.053106,0.127762
34440,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,...,0.5,0.103036,0.0572,0.144276,0.025572,0.080723,0.000002,0.02620,0.051904,0.110930


In [196]:
def generate_feature_set(df):
  popularity = popularity_normalization(df)
  genre = genre_tfidf(df)
  subjectivity_names = subjectivity(df)
  audio_features_df = audio_features(df)

  feature_set = pd.concat([genre, popularity, subjectivity_names, audio_features_df], axis = 1)
  # feature_set['id']=track_df['id'].values
  return feature_set

generate_feature_set(track_df)

,genre|21st,genre|432hz,genre|abstract,genre|acid,genre|acousmatic,genre|acoustic,genre|action,genre|adoracion,genre|adult,genre|adventista,...,mode|1,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.182996,0.1626,0.168547,0.025156,0.006245,0.001401,0.00942,0.162325,0.114421
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.156680,0.1676,0.178715,0.023701,0.005000,0.005025,0.04840,0.185170,0.130453
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.134413,0.1516,0.170210,0.043659,0.000478,0.000000,0.01196,0.140481,0.090525
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.180567,0.1428,0.171892,0.029314,0.040361,0.000047,0.01042,0.163727,0.092087
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.172672,0.1212,0.176541,0.014823,0.011265,0.000000,0.06260,0.131062,0.086421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34437,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.135425,0.0456,0.152570,0.014345,0.159036,0.013065,0.01888,0.080561,0.075718
34438,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.099798,0.1454,0.175155,0.045114,0.017530,0.000000,0.02580,0.057916,0.066813
34439,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.142105,0.1048,0.157060,0.016486,0.066667,0.011116,0.05960,0.053106,0.127762
34440,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.103036,0.0572,0.144276,0.025572,0.080723,0.000002,0.02620,0.051904,0.110930
